### Manipulation of PEAKS de novo results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/318_TROCAS7_Fusion_Apr2021_DENOVO_128/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/318_TROCAS7_Fusion_Apr2021_DENOVO_128


In [5]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks318 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/318_TROCAS7_Fusion_Apr2021_DENOVO_128/318_TROCAS7_Fusion_Apr2021_DENOVO_128_DN50.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks318))

print(peaks318.columns)

# These columns mess things up- get rid of them

del peaks318['Fraction']
del peaks318['Scan']
del peaks318['Source File']
del peaks318['Tag Length']
del peaks318['PTM']
del peaks318['tag (>=0%)']
del peaks318['mode']
del peaks318['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks318.columns = columns

mean_len = peaks318['length'].mean()
print(mean_len)

# look at the dataframe
peaks318.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 146
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
11.383561643835616


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,LSSPATLNSR,97,10,523.2861,2,47.12,79000000.0,1044.5564,1.3
1,LSSPATLNSR,96,10,523.2853,2,47.62,79000000.0,1044.5564,-0.4
2,LSSPATLNSR,96,10,523.2858,2,46.61,79000000.0,1044.5564,0.7
3,LATVLSPR,96,8,428.7668,2,61.60,50600000.0,855.5178,1.5
4,LATVLSPR,96,8,428.7661,2,62.61,50600000.0,855.5178,-0.2


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [6]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks318['A'] = peaks318['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks318['C'] = peaks318['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks318['D'] = peaks318['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks318['E'] = peaks318['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks318['F'] = peaks318['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks318['G'] = peaks318['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks318['H'] = peaks318['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks318 output, there will be no isoleucines (they're lumped in with leucines)
peaks318['I'] = peaks318['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks318['K'] = peaks318['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks318['L'] = peaks318['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks318['M'] = peaks318['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks318['N'] = peaks318['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks318['P'] = peaks318['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks318['Q'] = peaks318['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks318['R'] = peaks318['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks318['S'] = peaks318['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks318['T'] = peaks318['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks318['V'] = peaks318['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks318['W'] = peaks318['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks318['Y'] = peaks318['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks318['c-carb'] = peaks318['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks318['m-oxid'] = peaks318['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks318['n-deam'] = peaks318['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks318['q-deam'] = peaks318['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks318['stripped_peptide'] = peaks318['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks318['stripped_length'] = peaks318['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks318['ptm-total'] = peaks318['c-carb'] + peaks318['m-oxid'] + peaks318['n-deam'] + peaks318['q-deam']

# calculate NAAF numerator for each peptide k
peaks318['NAAF_num.'] = peaks318['Area'] / peaks318['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks318.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/318_MN_T24_GF_DN/318A_MN_T24_GF_DN50.csv")

# check out the results
peaks318.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,LSSPATLNSR,97,10,523.2861,2,47.12,79000000.0,1044.5564,1.3,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,7900000.0
1,LSSPATLNSR,96,10,523.2853,2,47.62,79000000.0,1044.5564,-0.4,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,7900000.0
2,LSSPATLNSR,96,10,523.2858,2,46.61,79000000.0,1044.5564,0.7,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,7900000.0
3,LATVLSPR,96,8,428.7668,2,61.60,50600000.0,855.5178,1.5,1,...,0,0,0,0,0,0,LATVLSPR,8,0,6325000.0
4,LATVLSPR,96,8,428.7661,2,62.61,50600000.0,855.5178,-0.2,1,...,0,0,0,0,0,0,LATVLSPR,8,0,6325000.0


In [7]:
# keep only stripped peptide I/L and NAAF
dn_318 = peaks318[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_318.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_318.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/318A_MN_T24_GF_DN50_ILnaafs.csv")

dn_318.head()

,stripped_peptide,Area,NAAF_num.
0,LSSPATLNSR,79000000.0,7900000.0
1,LSSPATLNSR,79000000.0,7900000.0
2,LSSPATLNSR,79000000.0,7900000.0
3,LATVLSPR,50600000.0,6325000.0
4,LATVLSPR,50600000.0,6325000.0


In [8]:
# made a new dataframe that contains the suMN of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks318['A'].sum(),
        'C': peaks318['C'].sum(),
        'D': peaks318['D'].sum(),
        'E': peaks318['E'].sum(),
        'F': peaks318['F'].sum(),
        'G': peaks318['G'].sum(),
        'H': peaks318['H'].sum(),
        'I': peaks318['I'].sum(),
        'K': peaks318['K'].sum(),
        'L': peaks318['L'].sum(),
        'M': peaks318['M'].sum(),
        'N': peaks318['N'].sum(),
        'P': peaks318['P'].sum(),
        'Q': peaks318['Q'].sum(),
        'R': peaks318['R'].sum(),
        'S': peaks318['S'].sum(),
        'T': peaks318['T'].sum(),
        'V': peaks318['V'].sum(),
        'W': peaks318['W'].sum(),
        'Y': peaks318['Y'].sum(),
        'c-carb': peaks318['c-carb'].sum(),
        'm-oxid': peaks318['m-oxid'].sum(),
        'n-deam': peaks318['n-deam'].sum(),
        'q-deam': peaks318['q-deam'].sum(),
        'Total area': peaks318['Area'].sum(),
        'Total length': peaks318['stripped_length'].sum()
       }

totalpeaks318 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks318['% C w/ carb'] = totalpeaks318['c-carb'] / totalpeaks318['C'] 

# calculate percentage of M's that are oxidized
totalpeaks318['% M w/ oxid'] = totalpeaks318['m-oxid'] / totalpeaks318['M'] 

# calculate percentage of N's that are deamidated
totalpeaks318['% N w/ deam'] = totalpeaks318['n-deam'] / totalpeaks318['N'] 

# calculate percentage of N's that are deamidated
totalpeaks318['% Q w/ deam'] = totalpeaks318['q-deam'] / totalpeaks318['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks318['NAAF denom.'] = totalpeaks318['Total area'] / totalpeaks318['Total length']

# write modified dataframe to new txt file
totalpeaks318.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/318_MN_T24_GF_DN//318A_MN_T24_GF_DN50_totals.csv")

totalpeaks318.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,130,85,47,57,22,124,46,0,107,199,...,21,8,2,2.389726e+09,1419,1.0,0.323077,0.096386,0.064516,1.684092e+06


In [9]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 1.684092e+06

# use NAAF >50% ALC to get NAAF factor
peaks318['NAAF factor'] = (peaks318['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksAAPTM_318 = peaks318[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_318['A-NAAF50'] = peaksAAPTM_318['A'] * peaks318['NAAF factor']
peaksAAPTM_318['C-NAAF50'] = peaksAAPTM_318['C'] * peaks318['NAAF factor']
peaksAAPTM_318['D-NAAF50'] = peaksAAPTM_318['D'] * peaks318['NAAF factor']
peaksAAPTM_318['E-NAAF50'] = peaksAAPTM_318['E'] * peaks318['NAAF factor']
peaksAAPTM_318['F-NAAF50'] = peaksAAPTM_318['F'] * peaks318['NAAF factor']
peaksAAPTM_318['G-NAAF50'] = peaksAAPTM_318['G'] * peaks318['NAAF factor']
peaksAAPTM_318['H-NAAF50'] = peaksAAPTM_318['H'] * peaks318['NAAF factor']
peaksAAPTM_318['I-NAAF50'] = peaksAAPTM_318['I'] * peaks318['NAAF factor']
peaksAAPTM_318['K-NAAF50'] = peaksAAPTM_318['K'] * peaks318['NAAF factor']
peaksAAPTM_318['L-NAAF50'] = peaksAAPTM_318['L'] * peaks318['NAAF factor']
peaksAAPTM_318['M-NAAF50'] = peaksAAPTM_318['M'] * peaks318['NAAF factor']
peaksAAPTM_318['N-NAAF50'] = peaksAAPTM_318['N'] * peaks318['NAAF factor']
peaksAAPTM_318['P-NAAF50'] = peaksAAPTM_318['P'] * peaks318['NAAF factor']
peaksAAPTM_318['Q-NAAF50'] = peaksAAPTM_318['Q'] * peaks318['NAAF factor']
peaksAAPTM_318['R-NAAF50'] = peaksAAPTM_318['R'] * peaks318['NAAF factor']
peaksAAPTM_318['S-NAAF50'] = peaksAAPTM_318['S'] * peaks318['NAAF factor']
peaksAAPTM_318['T-NAAF50'] = peaksAAPTM_318['T'] * peaks318['NAAF factor']
peaksAAPTM_318['V-NAAF50'] = peaksAAPTM_318['V'] * peaks318['NAAF factor']
peaksAAPTM_318['W-NAAF50'] = peaksAAPTM_318['W'] * peaks318['NAAF factor']
peaksAAPTM_318['Y-NAAF50'] = peaksAAPTM_318['Y'] * peaks318['NAAF factor']

# multiply the NAAF50 factor by the PTMN normalize its abundance by peak area and peptide length

peaksAAPTM_318['ccarb-NAAF50'] = peaksAAPTM_318['c-carb'] * peaksAAPTM_318['NAAF factor']
peaksAAPTM_318['moxid-NAAF50'] = peaksAAPTM_318['m-oxid'] * peaksAAPTM_318['NAAF factor']
peaksAAPTM_318['ndeam-NAAF50'] = peaksAAPTM_318['n-deam'] * peaksAAPTM_318['NAAF factor']
peaksAAPTM_318['qdeam-NAAF50'] = peaksAAPTM_318['q-deam'] * peaksAAPTM_318['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_318.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/318_MN_T24_GF_DN/318A_MN_T24_GF_DN50_naaf.csv")

peaksAAPTM_318.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,LSSPATLNSR,4.690955,1,0,0,0,0,0,0,0,...,4.690955,14.072865,4.690955,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,LSSPATLNSR,4.690955,1,0,0,0,0,0,0,0,...,4.690955,14.072865,4.690955,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,LSSPATLNSR,4.690955,1,0,0,0,0,0,0,0,...,4.690955,14.072865,4.690955,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,LATVLSPR,3.755733,1,0,0,0,0,0,0,0,...,3.755733,3.755733,3.755733,3.755733,0.0,0.0,0.0,0.0,0.0,0.0
4,LATVLSPR,3.755733,1,0,0,0,0,0,0,0,...,3.755733,3.755733,3.755733,3.755733,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMN

index = ['sample total']

data = {'NAAF': peaksAAPTM_318['NAAF factor'].sum(),
        'A': peaksAAPTM_318['A-NAAF50'].sum(),
        'C': peaksAAPTM_318['C-NAAF50'].sum(),
        'D': peaksAAPTM_318['D-NAAF50'].sum(),
        'E': peaksAAPTM_318['E-NAAF50'].sum(),
        'F': peaksAAPTM_318['F-NAAF50'].sum(),
        'G': peaksAAPTM_318['G-NAAF50'].sum(),
        'H': peaksAAPTM_318['H-NAAF50'].sum(),
        'I': peaksAAPTM_318['I-NAAF50'].sum(),
        'K': peaksAAPTM_318['K-NAAF50'].sum(),
        'L': peaksAAPTM_318['L-NAAF50'].sum(),
        'M': peaksAAPTM_318['M-NAAF50'].sum(),
        'N': peaksAAPTM_318['N-NAAF50'].sum(),
        'P': peaksAAPTM_318['P-NAAF50'].sum(),
        'Q': peaksAAPTM_318['Q-NAAF50'].sum(),
        'R': peaksAAPTM_318['R-NAAF50'].sum(),
        'S': peaksAAPTM_318['S-NAAF50'].sum(),
        'T': peaksAAPTM_318['T-NAAF50'].sum(),
        'V': peaksAAPTM_318['V-NAAF50'].sum(),
        'W': peaksAAPTM_318['W-NAAF50'].sum(),
        'Y': peaksAAPTM_318['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_318['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_318['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_318['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_318['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 1.684092e+06

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/318_MN_T24_GF_DN/318A_MN_T24_GF_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,176.626038,136.826821,20.580796,8.878911,3.243293,1.755624,16.217671,3.482343,0.0,8.299187,...,2.146206,20.580796,7.656234,0.079269,0.053204,1.0,0.662014,0.004266,0.04588,0.000105


## Export stripped peptides >50% ALC

In [11]:
###### keep only stripped peptide column 
pep50 = peaks318[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/318_MN_T24_GF_DN/318A_MN_T24_GF_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/318_MN_T24_GF_DN/318A_MN_T24_GF_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/318_MN_T24_GF_DN/318A_MN_T24_GF_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks318['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks318[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 146
# nonredundant stripped Peaks peptides >50% ALC 119
average peptide length Peaks peptides >50% ALC 9.719178082191782
# redundant Peaks peptides >50% ALC 146
# nonredundant Peaks peptides 122


,stripped_peptide
0,LSSPATLNSR
1,LSSPATLNSR
2,LSSPATLNSR
3,LATVLSPR
4,LATVLSPR
